In [ ]:
-- Active: 1733297022005@@127.0.0.1@3306@reasuransiindonesiautama

## **Create Table**

In [ ]:

DROP TABLE IF EXISTS `ifrs17_calculations`;

CREATE TABLE ifrs17_calculations (
    `Contract_ID` VARCHAR(255)
    ,`Product_ID` VARCHAR(255)
    ,`Start_Date` DATE
    ,`End_Date` DATE
    ,`First_Claim_Date` DATE
    ,`Last_Claim_Date` DATE
    ,`Claim_Count` INTEGER
    ,`Claim_Amount` DECIMAL(20, 2)
    ,`Premium` DECIMAL(20, 2)
    ,`Coverage` DECIMAL(20, 2)
    ,`Discount_Rate` DECIMAL(10, 2)
    ,`Future_Cash_Flow` DECIMAL(20, 2)
    ,`CSM` DECIMAL(30, 8)
);


## **Populate Table**

In [ ]:
INSERT INTO `ifrs17_calculations` (
    `Contract_ID`
)
SELECT
    `Contract_ID`
FROM `de_insurance_contracts`
UNION
SELECT
    `Contract_ID`
FROM `de_claims`

In [ ]:
UPDATE `ifrs17_calculations` `target`
INNER JOIN `de_insurance_contracts` `source` ON `target`.`Contract_ID` = `source`.`Contract_ID`
SET
    `target`.`Product_ID` = `source`.`Product_ID`
    ,`target`.`Start_Date` = `source`.`Start_Date`
    ,`target`.`End_Date` = `source`.`End_Date`
    ,`target`.`Premium` = `source`.`Premium`
    ,`target`.`Coverage` = `source`.`Coverage`
    ,`target`.`Discount_Rate` = `source`.`Discount_Rate`

In [ ]:
UPDATE `ifrs17_calculations` `target`
INNER JOIN (
    SELECT
        `Contract_ID`
        ,MIN(`Claim_Date`) `Min_Date`
        ,MAX(`Claim_Date`) `Max_Date`
    FROM `de_claims`
    GROUP BY `Contract_ID`
 ) `source` ON `target`.`Contract_ID` = `source`.`Contract_ID`
SET
    `target`.`Start_Date` = `source`.`Min_Date`
    ,`target`.`End_Date` = `source`.`Max_Date`
WHERE `target`.`Start_Date` IS NULL
AND `target`.`End_Date` IS NULL

In [ ]:
UPDATE `ifrs17_calculations` `target`
INNER JOIN (
    SELECT
        `base`.`Contract_ID`
        ,MIN(`source`.`Claim_Date`) `First_Claim_Date`
        ,MAX(`source`.`Claim_Date`) `Last_Claim_Date`
        ,COUNT(`source`.`Claim_ID`) `Claim_Count`
        ,SUM(`source`.`Claim_Amount`) `Claim_Amount`
    FROM `ifrs17_calculations` `base`
    INNER JOIN `de_claims` `source`
        ON `base`.`Contract_ID` = `source`.`Contract_ID`
        AND COALESCE(`base`.`Start_Date`, `source`.`Claim_Date`) <= `source`.`Claim_Date`
        AND COALESCE(`base`.`End_Date`, `source`.`Claim_Date`) >= `source`.`Claim_Date`
    GROUP BY
        `base`.`Contract_ID`
) `source` ON `target`.`Contract_ID` = `source`.`Contract_ID`
SET
    `target`.`First_Claim_Date` = `source`.`First_Claim_Date`
    ,`target`.`Last_Claim_Date` = `source`.`Last_Claim_Date`
    ,`target`.`Claim_Count` = `source`.`Claim_Count`
    ,`target`.`Claim_Amount` = `source`.`Claim_Amount`

Tidak ada kontrak yang sudah diklaim.

In [ ]:
UPDATE `ifrs17_calculations` `target`
SET
    `target`.`Future_Cash_Flow` = `target`.`Coverage` - COALESCE(`target`.`Claim_Amount`, 0.00)

In [ ]:
UPDATE `ifrs17_calculations` `target`
SET
    `target`.`CSM` = `target`.`Future_Cash_Flow` * `Discount_Rate`